# $PSO_{mv}$ implementation #

In this notebook it is presented a **naive implementation** of the algorithm described in F. Wang, H. Zhang, A. Zhou *A particle swarm optimization algorithm for mixed-variable optimization problems* which combines two variants of the PSO algorithm to evolve both the discrete and the continuous part of a swarm to tackle mixed discrete-integer optimization problems. This implementation is used to compare the optimization results of our algorithm.

Main ideas behind the algorithm according to the paper

<img src="./images/pso_mv.png" alt="alternative text" width="950"/>

## Code ##

In [149]:
import math 
import numpy as np
import copy
import matplotlib.pyplot as plt
import random

In [150]:
%run 02_GA-PSO_on_artificial_test_functions.ipynb

### Continuous part ##

In [151]:
def continuous_PSO(element, index, velocity, population, particle_best, d_discr, d_cont, w, minimum, maximum, cog = 2):
    '''
    Performs the Continuous Reproduction Method described above
    '''
    
    #all arrays are assumed sorted according to their particle_best value
    N = len(population)
    
    for j in range(d_discr,d_discr+d_cont):
        r = np.random.randint(index,N)
        randm = np.random.uniform(0,1)
        
        #update velocity
        velocity[j-d_discr]=w[index]*velocity[j-d_discr]+cog*randm*(particle_best[r][j]-element[j])
        
        #update position
        element[j]+=velocity[j-d_discr]
        
        #boundaries
        if element[j]>maximum:
            element[j]=maximum
            velocity[j-d_discr]=-(velocity[j-d_discr])
        if element[j]<minimum:
            element[j]=minimum
            velocity[j-d_discr]=-(velocity[j-d_discr])
            
                        
    #return updated element and its velocity
    return element, velocity 

In [152]:
def discrete_PSO(element, index, prob, population, particle_best, d_discr,d_cont, alpha, num_values, poss_values):
    N = len(population)
    count = np.zeros((d_discr, num_values))

    #convert each list in particle_best to a numpy array and stack them vertically
    particle_best_np = np.vstack([np.array(lst) for lst in particle_best])
    
    const_prob=np.array([[1/num_values] * num_values for _ in range(d_discr)])

    for j in range(d_discr):
        count[j] = np.sum(np.isclose(particle_best_np[int(N/2):, j][:, None], poss_values[None, :]), axis=0)
        count_int = count[j].astype(int)

        prob_np = np.array(prob[j])
        const_prob_values = np.array([1/num_values] * num_values) 
               
        #ensure prob[j] is a NumPy array before performing arithmetic operations
        prob[j] = alpha[index] * prob_np + (0.8 - alpha[index]) * count_int / (N/2)+0.2*const_prob_values
        

    #update only the first d_discr values in element according to the probabilities obtained
    element[:d_discr] = [poss_values[np.random.choice(num_values, p=prob[j])] for j in range(d_discr)]

    return element, prob

    

In [153]:
from scipy.stats import cauchy

def tuning(w_avg,alpha_avg):
    '''
    w_avg and alpha_avg are the average value of historical best parameters
    '''
    
    rand= np.random.uniform(0,1)
    
    if rand<0.5:
        w = cauchy.rvs(loc=w_avg, scale=0.1, size=1)[0]
        alpha = cauchy.rvs(loc=alpha_avg, scale=0.1, size=1)[0]
        if alpha<=0:
            alpha=0
        if alpha>=0.8:
            alpha=0.8
        if w<0:
            w=0
        if w>1.5:
            w=1.5
        
    else:         
        w = np.random.normal(loc=w_avg, scale=math.sqrt(0.1))
        alpha = np.random.normal(loc=alpha_avg, scale=math.sqrt(0.1))
        if alpha<=0:
            alpha=0
        if alpha>=0.8:
            alpha=0.8
        if w<0:
            w=0
        if w>1.5:
            w=1.5    
        
    return w, alpha

In [154]:
def pop_sorting(particle_number,pop, vals, best_pos, best_vals,velocities,w,alpha,w_avg,alpha_avg):
    #sort best_vals in decreasing order
    sorted_indices = sorted(range(len(best_vals)), key=lambda k: best_vals[k], reverse=True)

    #sort everything based on the sorted order of best_vals
    pop = [pop[i] for i in sorted_indices]
    vals = [vals[i] for i in sorted_indices]
    best_pos = [best_pos[i] for i in sorted_indices]
    particle_number = [particle_number[i] for i in sorted_indices]
    velocities = [velocities[i] for i in sorted_indices]
    w = [w[i] for i in sorted_indices]
    alpha = [alpha[i] for i in sorted_indices]
    w_avg = [w_avg[i] for i in sorted_indices]
    alpha_avg = [alpha_avg[i] for i in sorted_indices]
    best_vals = sorted(best_vals, reverse=True)

    return particle_number,pop, vals, best_pos, best_vals, velocities, w, alpha, w_avg,alpha_avg

In [155]:
def PSO_mv(epochs,function,rot_matrix,n_pop,d_discr,d_cont,minimum,maximum,granularity,max_fitness_eval=100000):
    '''
    Main function for the PSO_mv algorithm
    '''
    
    #initialization
    population = np.array(initialization(n_pop,d_discr,d_cont,minimum,maximum,granularity))     
    
    #number of fitness evaluation
    f_eval = 0
    
    #evaluation
    values, f_eval = evaluation(function,population,f_eval,d_discr,d_cont,rot_matrix,max_fitness_eval)
    values=np.array(values)
    
       
    particle_number = np.array(list(range(n_pop)))
    
    #intialize particle_best, at the beginning
    particle_best_position= np.array(copy.copy(population))
    particle_best_values = np.array(copy.copy(values))
    
    #initial velocities for the continuous part
    velocities = np.array([[0] * d_cont for _ in range(n_pop)])
        
     
    #initial probability for the discrete part
    probabilities=np.array([[1/granularity] * granularity for _ in range(d_discr)])
    
    #initialize w and alphas
    w = [0.5] * n_pop
    alpha = [0.5] * n_pop
    
    w_avg = [0.5] * n_pop
    alpha_avg = [0.5] * n_pop
    
    #sorting population according to their particle_best value
    particle_number, population, values, particle_best_position, particle_best_values,velocities,w,alpha,w_avg,alpha_avg = pop_sorting(
                                                                                        particle_number, population,
                                                                                        values, particle_best_position, 
                                                                                        particle_best_values,velocities,
                                                                                        w,alpha,w_avg,alpha_avg)
    
    
    #values ordered according to particle_number
    w_hist = [[0.5] for _ in range(n_pop)]
    alpha_hist = [[0.5] for _ in range(n_pop)]
    
    tot_best=min(particle_best_values)
    
    bmrk=[tot_best]
    f_evaluations=[f_eval]
     
      
    for epoch in range(0,epochs):
        
        sort_ind = sorted(range(len(particle_number)), key=lambda k: particle_number[k], reverse=False)

        prev_values = [values[i] for i in sort_ind]
        
        #extract the w and alpha parameter
        for q in range(0,n_pop):
            w[q], alpha[q] = tuning(w_avg[q],alpha_avg[q])              
            
        if min(particle_best_values)<tot_best:
                tot_best= min(particle_best_values)
            
        if epoch%15==0:
            f_evaluations+=[f_eval]            
            bmrk+=[tot_best]
            
        #continuous PSO
        for i in range(0,n_pop):
            ind=np.where(np.array(particle_number)==i)[0][0]
            
            #continuous part
            population[ind],velocities[ind] = continuous_PSO(population[ind], ind, velocities[ind], population, particle_best_position, 
                                                       d_discr, d_cont, w, minimum, maximum, cog = 2)
                        
            
            #discrete part
            population[ind],probabilities = discrete_PSO(population[ind], ind, probabilities, population, particle_best_position, 
                                                       d_discr, d_cont, alpha, granularity , 
                                                       np.linspace(minimum, maximum, granularity))
                       
            f_eval+=1
            #calculate the value
            values[ind] = function(population[ind][:d_discr],population[ind][-d_cont:],d_discr+d_cont,rot_matrix)
            #substitute in particle_best if it is better
            if values[ind] < particle_best_values[ind]:
                particle_best_values[ind] = values[ind]
                particle_best_position[ind] = population[ind]
            
            #sort everything
            particle_number, population, values, particle_best_position, particle_best_values,velocities,w,alpha,w_avg,alpha_avg = pop_sorting(
                                                                                        particle_number, population,
                                                                                        values, particle_best_position,
                                                                                        particle_best_values,velocities,w,alpha,w_avg,alpha_avg)
        #exit if too many function evaluations
        if f_eval>max_fitness_eval:
            return bmrk,f_evaluations          
        
        
        
        #procedure to tune parameters
        #values ordered according to particle_number
        sort_ind = sorted(range(len(particle_number)), key=lambda k: particle_number[k], reverse=False)
        vals = [values[i] for i in sort_ind]
        for i in range(0,n_pop):
            k=np.where(np.array(particle_number)==i)[0][0]
            if vals[i]<prev_values[i]:
                w_hist[i]+=[w[k]]
                alpha_hist[i]+=[alpha[k]]
            w_avg[k]=np.mean(w_hist[i])
            alpha_avg[k]=np.mean(alpha_hist[i])
    return bmrk,f_evaluations

In [156]:
#example of use
#PSO_mv(500,alpine1_func_rotated,generate_random_rotation_matrix(50),50,25,25,-10,10,21,max_fitness_eval=100000)